This document contains the codes that preprocess the data for regression analysis and build model of linear regression using variables from the integrated datasets. 

Dependent variable: prevalence of obesity

The contents include:

i. Preprocessing data
ii. construct linear regression model
iii. collinearity
iv. Detecting outliers and high leverage points
v. residual diagnosis: 
     a. do errors follow normal distribution? 
     b. is mean of errors zero?
     c. do errors have the same variance?
     d. are errors independent of each other?
vi. model assessment and interpretation

In [4]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
#read the complete dataset
url='https://raw.githubusercontent.com/cathyxinxyz/Capstone_Project_1/master/Datasets/Combined_data.csv'
df=pd.read_csv(url,index_col='FIPS',encoding="ISO-8859-1")

In [6]:
#standardize data: (x-mean(x))/std
from sklearn import preprocessing
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

scaled_columns=[c for c in df.columns if is_numeric_dtype(df[c])]
df_for_scale=df[scaled_columns].dropna()
X=df_for_scale.values
scaler=  preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)
scaled_df = pd.DataFrame(X_scaled, index=df_for_scale.index, columns=df_for_scale.columns)

In [7]:
#replace space in variable names with _
scaled_df.rename(columns=lambda x:'above'.join(x.split('>=')), inplace=True)
scaled_df.rename(columns=lambda x:'under'.join(x.split('<')), inplace=True)
scaled_df.rename(columns=lambda x:'_'.join(x.split(' ')), inplace=True)

In [8]:
vars=list(scaled_df.columns.difference(['prevalence_of_diabetes',
                                             'prevalence_of_obesity',
                                             'Adult_db', 
                                             'Adult_ob']))
features='+'.join(vars)
y, X = dmatrices('prevalence_of_obesity ~1+' + features, scaled_df, return_type='dataframe')

mod = smf.OLS(y, X)
result=mod.fit()

In [9]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     prevalence_of_obesity   R-squared:                       0.616
Model:                               OLS   Adj. R-squared:                  0.613
Method:                    Least Squares   F-statistic:                     182.3
Date:                   Sat, 30 Dec 2017   Prob (F-statistic):               0.00
Time:                           09:20:24   Log-Likelihood:                -2913.6
No. Observations:                   3098   AIC:                             5883.
Df Residuals:                       3070   BIC:                             6052.
Df Model:                             27                                         
Covariance Type:               nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                         -7.459e-16      0.011  -6.67e-14      1.000      -0.022       0.022
American_Indian_or_Alaska_Native      0.2020      0.100      2.023      0.043       0.006       0.398
Asian                                -0.0550      0.037     -1.485      0.138      -0.128       0.018
Assistance                            0.0469      0.013      3.675      0.000       0.022       0.072
Black                                 0.4379      0.184      2.386      0.017       0.078       0.798
Convenience                          -0.0130      0.014     -0.958      0.338      -0.040       0.014
Farm                                  0.0056      0.012      0.455      0.649      -0.018       0.030
Fast_food                            -0.0365      0.014     -2.640      0.008      -0.064      -0.009
Full_service                         -0.0763      0.016     -4.748      0.000      -0.108      -0.045
Grocery                              -0.0586      0.016     -3.613      0.000      -0.090      -0.027
Hawaiian_or_Pacific_Islander          0.0410      0.019      2.207      0.027       0.005       0.077
Hispanic                              0.0703      0.168      0.419      0.675      -0.258       0.399
Low_Access_Child                     -0.1042      0.060     -1.724      0.085      -0.223       0.014
Low_Access_Overall                    0.1752      0.088      1.998      0.046       0.003       0.347
Low_Access_Seniors                   -0.1247      0.045     -2.777      0.006      -0.213      -0.037
Low_Access_low_income                 0.0333      0.035      0.948      0.343      -0.036       0.102
Low_insecurity                        0.0764      0.034      2.236      0.025       0.009       0.143
Poverty_rate                          0.0070      0.020      0.349      0.727      -0.032       0.046
Recreation_facility                  -0.0409      0.013     -3.208      0.001      -0.066      -0.016
SNAP_store                            0.0334      0.016      2.054      0.040       0.002       0.065
Specialized                           0.0038      0.012      0.309      0.757      -0.020       0.028
Supercenter                           0.0477      0.012      3.949      0.000       0.024       0.071
Very_low_insecurity                  -0.0357      0.035     -1.033      0.302      -0.103       0.032
WIC_store                             0.0446      0.016      2.744      0.006       0.013       0.076
White                                 0.3847      0.251      1.534      0.125      -0.107       0.876
above65                              -0.0227      0.022     -1.049      0.294      -0.065       0.020
prevalence_of_physical_inactivity     0.4976      0.016     30.185      0.000       0.465   

In [10]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [11]:
vif

,VIF Factor,features
0,1.000000,Intercept
1,79.727772,American_Indian_or_Alaska_Native
2,10.950420,Asian
3,1.300721,Assistance
4,269.274453,Black
5,1.471712,Convenience
6,1.205969,Farm
7,1.528162,Fast_food
8,2.061382,Full_service
9,2.103192,Grocery


The results incidate a strong multicollinearity among variables that quantify 
the race/ethnicity composition: White, Black, Hispanic and American_Indian_or_Alaska_Native
as indicated in the section of exploratory data analysis, the percentage of White people in the population
negatively correlates with percentage of Black people, Hispanic people, or American_Indian_or_Alaska_Native people

In addition, there is also moderate multicollinearity among variables Low_Access_Overall,Low_Access_Child, Low_Access_Seniors, Low_Access_Overall, Low_Access_low_income, as indicated in exploratory analysis

Lastly, low_security and very_low_security have a moderate level of 
multicollinearity, which is reasonable since very low food security is a subcategory of low food security.

In order to remove the variables with high multicollinearity, I remove variables Black, Hispanic and American_Indian_or_Alaska_Native, Asian, Low_Access_Child, Low_Access_Seniors, Low_Access_low_income, Very_low_insecurity


In [12]:
vars=list(set(vars).difference(set(['Black','Hispanic','Asian','American_Indian_or_Alaska_Native',
                                    'Low_Access_Child','Low_Access_Seniors','Low_Access_low_income',
                                    'Very_low_insecurity'])))

In [15]:
features='+'.join(vars)
y, X = dmatrices('prevalence_of_obesity ~1+' + features, scaled_df, return_type='dataframe')

mod = sm.OLS(y, X)
result=mod.fit()

In [16]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     prevalence_of_obesity   R-squared:                       0.557
Model:                               OLS   Adj. R-squared:                  0.554
Method:                    Least Squares   F-statistic:                     203.4
Date:                   Sat, 30 Dec 2017   Prob (F-statistic):               0.00
Time:                           09:25:56   Log-Likelihood:                -3135.9
No. Observations:                   3098   AIC:                             6312.
Df Residuals:                       3078   BIC:                             6433.
Df Model:                             19                                         
Covariance Type:               nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                         -7.858e-16      0.012  -6.55e-14      1.000      -0.024       0.024
SNAP_store                            0.0710      0.017      4.128      0.000       0.037       0.105
Poverty_rate                          0.0636      0.019      3.350      0.001       0.026       0.101
Fast_food                            -0.0515      0.015     -3.518      0.000      -0.080      -0.023
Recreation_facility                  -0.0230      0.014     -1.698      0.090      -0.050       0.004
prevalence_of_physical_inactivity     0.5902      0.017     35.272      0.000       0.557       0.623
above65                              -0.1001      0.017     -5.750      0.000      -0.134      -0.066
Low_insecurity                        0.0338      0.013      2.579      0.010       0.008       0.060
Full_service                         -0.0953      0.017     -5.605      0.000      -0.129      -0.062
Low_Access_Overall                    0.0065      0.013      0.487      0.626      -0.020       0.033
Assistance                            0.0309      0.013      2.325      0.020       0.005       0.057
Convenience                           0.0229      0.014      1.607      0.108      -0.005       0.051
Supercenter                           0.0446      0.013      3.454      0.001       0.019       0.070
White                                 0.0530      0.017      3.051      0.002       0.019       0.087
under18                               0.0643      0.016      4.037      0.000       0.033       0.096
Grocery                              -0.0657      0.017     -3.806      0.000      -0.100      -0.032
Specialized                           0.0140      0.013      1.075      0.283      -0.012       0.039
Hawaiian_or_Pacific_Islander         -0.0220      0.012     -1.789      0.074      -0.046       0.002
WIC_store                             0.0511      0.017      3.071      0.002       0.018       0.084
Farm                                  0.0163      0.013      1.239      0.215      -0.009       0.042
==============================================================================
Omnibus:                       20.516   Durbin-Watson:                   1.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.359
Skew:                           0.174   Prob(JB):                     2.30e-05
Kurtosis:                       3.210   Cond. No.                         3.69
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

After removing the variables that have collinearity with other variables, R square decreases from 0.616 to 0.557, which is reasonable since I only use a subset of the variables in the model. In addition, coefficients of several variables become statistically significant (p value falls under 0.05). 

In [17]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [18]:
vif

,VIF Factor,features
0,1.000000,Intercept
1,2.056158,SNAP_store
2,2.501780,Poverty_rate
3,1.487055,Fast_food
4,1.271987,Recreation_facility
5,1.943646,prevalence_of_physical_inactivity
6,2.103695,above65
7,1.195222,Low_insecurity
8,2.006019,Full_service
9,1.240966,Low_Access_Overall


vif test shows that there is no sever multicollinearity this time

The regression results indicate that there are several variables whose coefficients are not statistically significant (p value >0.05). Therefore, I remove the insignificant independent variables ('Recreation_facility', 'Low_Access_Overall' and 'Convenience', 'Specialized', 'Hawaiian_or_Pacific_Islander', 'Farm')

In [19]:
vars=list(set(vars).difference(set(['Recreation_facility', 'Low_Access_Overall','Convenience', 'Specialized', 'Hawaiian_or_Pacific_Islander', 'Farm'])))

In [22]:
features='+'.join(vars)
y, X = dmatrices('prevalence_of_obesity ~1+' + features, scaled_df, return_type='dataframe')

mod = smf.OLS(y, X)
result=mod.fit()

In [23]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     prevalence_of_obesity   R-squared:                       0.555
Model:                               OLS   Adj. R-squared:                  0.553
Method:                    Least Squares   F-statistic:                     295.9
Date:                   Sat, 30 Dec 2017   Prob (F-statistic):               0.00
Time:                           09:31:59   Log-Likelihood:                -3141.6
No. Observations:                   3098   AIC:                             6311.
Df Residuals:                       3084   BIC:                             6396.
Df Model:                             13                                         
Covariance Type:               nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                         -7.858e-16      0.012  -6.54e-14      1.000      -0.024       0.024
SNAP_store                            0.0812      0.017      4.917      0.000       0.049       0.114
Poverty_rate                          0.0687      0.019      3.667      0.000       0.032       0.105
Fast_food                            -0.0582      0.014     -4.058      0.000      -0.086      -0.030
prevalence_of_physical_inactivity     0.5952      0.016     36.511      0.000       0.563       0.627
above65                              -0.0909      0.017     -5.318      0.000      -0.124      -0.057
Low_insecurity                        0.0337      0.013      2.569      0.010       0.008       0.059
Full_service                         -0.0834      0.016     -5.211      0.000      -0.115      -0.052
Assistance                            0.0349      0.013      2.667      0.008       0.009       0.061
Supercenter                           0.0433      0.013      3.360      0.001       0.018       0.069
White                                 0.0567      0.017      3.360      0.001       0.024       0.090
under18                               0.0692      0.016      4.372      0.000       0.038       0.100
Grocery                              -0.0618      0.017     -3.623      0.000      -0.095      -0.028
WIC_store                             0.0517      0.017      3.129      0.002       0.019       0.084
==============================================================================
Omnibus:                       20.127   Durbin-Watson:                   1.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.969
Skew:                           0.172   Prob(JB):                     2.80e-05
Kurtosis:                       3.211   Cond. No.                         3.38
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

After removing insignificant independent variables, R square slightly decreases from 0.557 to 0.555, indicating removing these variables does not severely reduce model fit. The next step is to check any possible interactions. 

In [24]:
from itertools import combinations
vars_with_interaction=list()
vars_with_interaction.extend(vars)
for var_pair in combinations(vars,2):
    scaled_df[var_pair[0]+'_'+var_pair[1]]=scaled_df[var_pair[0]]*scaled_df[var_pair[1]]
    vars_with_interaction.append(var_pair[0]+'_'+var_pair[1])

In [25]:
features='+'.join(vars_with_interaction)
y, X = dmatrices('prevalence_of_diabetes ~1+' + features, scaled_df, return_type='dataframe')

mod = smf.OLS(y, X)
result=mod.fit()

In [26]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     prevalence_of_diabetes   R-squared:                       0.730
Model:                                OLS   Adj. R-squared:                  0.722
Method:                     Least Squares   F-statistic:                     89.42
Date:                    Sat, 30 Dec 2017   Prob (F-statistic):               0.00
Time:                            09:33:33   Log-Likelihood:                -2366.3
No. Observations:                    3098   AIC:                             4917.
Df Residuals:                        3006   BIC:                             5472.
Df Model:                              91                                         
Covariance Type:                nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            0.0156      0.016      0.980      0.327      -0.016       0.047
SNAP_store                                           0.1395      0.017      8.011      0.000       0.105       0.174
Poverty_rate                                         0.1247      0.019      6.558      0.000       0.087       0.162
Fast_food                                            0.0224      0.014      1.592      0.111      -0.005       0.050
prevalence_of_physical_inactivity                    0.5264      0.014     36.376      0.000       0.498       0.555
above65                                              0.1911      0.018     10.592      0.000       0.156       0.227
Low_insecurity                                       0.1002      0.012      8.271      0.000       0.076       0.124
Full_service                                        -0.1869      0.022     -8.461      0.000      -0.230      -0.144
Assistance                                          -0.1579      0.011    -13.770      0.000      -0.180      -0.135
Supercenter                                         -0.0134      0.013     -1.063      0.288      -0.038       0.011
White                                               -0.1011      0.017     -6.039      0.000      -0.134      -0.068
under18                                             -0.0566      0.017     -3.286      0.001      -0.090      -0.023
Grocery                                             -0.0796      0.022     -3.622      0.000      -0.123      -0.036
WIC_store                                           -0.0557      0.023     -2.473      0.013      -0.100      -0.012
SNAP_store_Poverty_rate                              0.0043      0.018      0.233      0.816      -0.032       0.040
SNAP_store_Fast_food                                 0.0040      0.014      0.281      0.779      -0.024       0.032
SNAP_store_prevalence_of_physical_inactivity         0.0029      0.018      0.164      0.869      -0.032       0.038
SNAP_store_above65                                  -0.0259      0.019     -1.359      0.174      -0.063       0.011
SNAP_store_Low_insecurity                            0.0312      0.016      1.900      0.058      -0.001       0.063
SNAP_store_Full_service                             -0.0046      0.016     -0.283      0.777      -0.037       0.027
SNAP_store_Assistance                               -0.0277      0.014     -1.951      0.051      -0.056       0.000
SNAP_store_Supercenter                               0.0333      0.014      2.390      0.017       0.006       0.061
SNAP_store_White                                     0.0047      0.017      0.282      0.778      -0.028       0.0

After including all the possible interaction terms, R square increases from 0.555 to 0.730, which suggests improvement in model fitting. However, there are several dramatic changes: the sign of coefficients of variables White